# Import

In [1]:
import csv
import os
import pickle

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_mutual_info_score

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
tqdm.pandas()

# Read data

In [3]:
newsgroups_df = pd.read_csv("../../../Preprocessing/data/20News/master.csv", index_col=0)

In [4]:
with open("../../../Preprocessing/data/20News/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [5]:
vector_sizes = [2, 3, 4, 6, 7, 8, 9, 10, 20, 40, 80, 160, 320, 640]

In [6]:
label = newsgroups_df["class"].to_numpy()

In [7]:
model_nums = 2
vector_model_num = 0

In [8]:
def gmm_value(gmm, vectors, label):
    pred = gmm.predict(vectors)
    aic = gmm.aic(vectors)
    bic = gmm.bic(vectors)
    mi = adjusted_mutual_info_score(pred, label)
    logl = gmm.score(vectors, label)
    return {"pred": pred, "aic":aic, "bic":bic, "mi":mi, "logl":logl}

In [9]:
vectors_path = "../../../Clustering/data/20News/doc2vec/vector/"
models_path = "../../../Clustering/data/20News/doc2vec/GMM/model/"
pred_path = "../../../Clustering/data/20News/doc2vec/GMM/pred/"
stats_list = []
for model_num in range(model_nums):
    stats = []
    for vector_size in tqdm(vector_sizes):
        vector_path = f"{vectors_path}{vector_size}/normalized/{vector_model_num}.csv"
        vectors = np.loadtxt(vector_path, delimiter=",")
        
        gmm = pickle.load(open(f"{models_path}{vector_size}/{model_num}.sav", "rb"))
        value = gmm_value(gmm, vectors, label)
        pred, *stat = value.items()
        stats.append(dict(stat))
    stats_list.append(stats)

100%|██████████| 14/14 [32:30<00:00, 139.31s/it]


In [13]:
stats_list = [pd.DataFrame(stats, index=vector_sizes) for stats in stats_list]

In [14]:
stats_path = "../../data/20News/doc2vec/GMM/stats/"
for model_num in range(model_nums):
    os.makedirs(f"{stats_path}", exist_ok=True)
    stats_list[model_num].to_csv(f"{stats_path}{model_num}.csv")

In [15]:
stats_list[0]

,aic,bic,mi,logl
2,-9.450630e+04,-9.357286e+04,0.111436,2.513645
3,-1.274552e+04,-1.118455e+04,0.165213,0.348708
4,-1.132760e+04,-8.982228e+03,0.238775,0.316396
6,4.141376e+02,4.798965e+03,0.359643,0.018674
7,-5.892896e+03,-2.530197e+02,0.395102,0.194495
8,-9.125519e+03,-2.073713e+03,0.392004,0.289810
9,-2.492829e+04,-1.630767e+04,0.399538,0.719683
10,-3.320079e+04,-2.285448e+04,0.421546,0.950833
20,-2.492712e+05,-2.130395e+05,0.456065,6.858463
40,-9.213421e+05,-7.862753e+05,0.478621,25.357637
